In [1]:
from sklearn.linear_model import LinearRegression

## Principles of supervized Machine Learning

本笔记本介绍了**监督式机器学习**，重点关注回归任务。它解释了机器学习模型的概念，作为一个根据解释变量 $X^1, \dots, X^p$ 预测目标变量 $y$ 的函数，并将学习过程描述为一个**经验风险最小化问题**。笔记本介绍了梯度下降和随机梯度下降作为寻找最小化预测误差的模型参数的迭代方法。

接下来通过 Python 的 `sklearn` 库演示了线性回归的实际例子，随后讨论了不同的回归变体，如Ridge回归、Lasso回归和ElasticNet，这些变体引入了正则化技术以防止过拟合并改善模型的泛化能力。此外，笔记本还包含一个练习，要求使用梯度下降手动计算回归参数。

### What is a machine learning model ?

机器学习的应用通常是从以下情况开始的：我们考虑一个无法直接解释或观察的变量 $y$，但我们**假设**它可以通过另一组可观察的变量 $X^1$, ..., $X^p$ 来解释或推导。我们称这些变量为**解释变量**（explanatory variables），或**特征**（features），或**描述符**（descriptors），而 $y$ 通常称为**目标变量**（target variable）。

目标变量可以是里昂市明天的平均气温，在这种情况下，一组强有力的可观察变量是当天以及过去七天在该市及其邻近城市和村庄的平均气温。请注意，**今天**的温度和**明天**的温度是不同的变量：前者在任何一天都可以观察到，而后者永远不能直接观察到。

给定一组解释变量 $X^1$, ..., $X^p$ 和目标变量 $y$，我们的目标是**找到函数** $\hat{y} = F(X^1, ..., X^p)$，使其能够**最接近地近似目标变量** $y$。因此，本质上，机器学习模型（machine learning model）只是一个将由一组特征 $x^1$, ..., $x^p$ 描述的数据点 $x$ 转换为未知的目标变量 $y$ 的值的函数。由于通常没有线索表明 $y$ 如何依赖于选定的解释变量，**监督式机器学习**（supervised machine learning）的方法是**基于一组历史数据来学习这种函数**。也就是说，给定一组 $n$ 个数据点 $x_{1} = (x_1^{1}, ..., x^p_{1})$ 及其对应的目标值 $y_{1}$，直到数据点 $x_{n} = (x^1_{n}, ..., x^p_{n})$ 及其对应的目标值 $y_{n}$，我们希望找到一个函数 $F$，使得 $F(x^1_{i}, ..., x^p_{i})$ 能够最好地近似每个索引 $i \leq n$ 的 $y_{i}$。

总结来说，"监督式机器学习"（supervised machine learning）的定义包含以下几点：
- **学习**（learning）：模型不是通常意义上的程序，而是以**数据驱动**（data-driven）的方式构建的：我们不是从单一模型开始，而是从一个**元模型**（meta-model）开始，一个无限的模型集合，接下来会执行一个**学习阶段**（learning phase），以便从这一大堆模型中选出一个在特定数据样本上预测能力最优的模型。
- **机器**（machine）：在元模型中选择哪个模型的过程是**自动化的**（automatic），并基于明确的**优化逻辑**（optimization logic）。
- **监督**（supervised）：此学习过程的目的是为了最优地预测一组**已知的**（already known）目标值。

数据科学（Data Science）中的大多数用例涉及两种任务：
- **回归**（regression）：目标变量 $y$ 是**连续的**（continuous），例如其值是实数或向量。
- **分类**（classification）：目标变量 $y$ 是**类别型的**（categorical），例如其值属于一个有限的集合。

有些情况可能涉及两者（例如，待预测的变量是连续和类别变量的组合），但在所有情况下，问题都可以分解为独立的任务。

### Learning as empirical risk minimization

让我们描述一下学习阶段的一般公式化，也称为**拟合过程**（fitting process），以回归问题为例。

如前所述，机器学习模型是元模型（meta-model）中的无限多种模型可能性之一。我们通常将元模型表示为一个带参数的量 `Model(w)`，其参数集合为 $w \in W = \mathbb{R}^d$，即**可学习参数**（learnable parameters）或**自由参数**（free parameters）。此处的维度 $d$ 是模型的可学习参数的数量。然后，单一选择 $w = (w^1, ..., w^d) \in \mathbb{R}^d$ 会产生一个通常意义上的模型：
$$
model = Model(w)
$$
当调用模型对某个数据点 $x$ 进行预测时，输出值通常表示为 $\widehat{y}$，或者为了避免歧义表示为 `model.predict(x)`。

由于监督学习（supervised learning）是基于现有数据的，令 $D$ 表示这一数据集，它是一组 $(x, y)$ 对，其中 $x$ 是由一些解释变量（explanatory variables）描述的数据点，$y$ 是我们希望从 $x$ 推断出的目标（连续）值。

给定由参数集合 $W$ 参数化的元模型 `Model` 和一个选择 $w \in \mathbb{R}^d$，我们通常通过**均方根误差**（Root Mean Squared Error, RMSE）来度量模型 `Model(w)` 在数据集 $D$ 上的**经验误差**（empirical error），也称为**经验风险**（empirical risk）或**经验损失**（empirical loss）：
$$
\text{RMSError}\left(\text{Model}(w), D \right) = \frac{1}{n} \sqrt{\sum_{(x, y) \in D} \left( \text{Model}(w).\text{predict}(x) - y \right) ^2}
$$
注意，当且仅当 `Model(w).predict(x)` 对数据集中的所有点都与期望值 $y$ 完全匹配时，模型的经验误差为0。接下来，**学习阶段**（learning phase）就是要找到一个经验风险最小化问题的解或近似解，即找到一个接近最优解的 $\hat{w} \in \mathbb{R}^d$，其中最优解是：
$$
w^* = \arg \min _{w \in \mathbb{R}^d} \text{RMSError}\left(\text{Model}(w), D \right)
$$

<a id="sgd"></a>

### Approximate minimizer using Stochastic Gradient Descent

在实际应用中，测试每一个参数选择并找到最优解是不可能的，因为参数空间 $\mathbb{R}^d$ 是无限的，而且我们通常没有问题的确切解的表达式。因此，通常的做法是**迭代地**计算一系列选择，使得经验误差（empirical error）逐渐减小，直到其收敛于一个选择 $\hat{w}$，这个最终选择将作为我们最好的参数。

这一方法的数学有效性解释在于函数
$$
E(w) := Error(Model(w), D)
$$
在可学习参数（learnable parameters）空间 $W = \mathbb{R}^d$ 上被适当地选择，使得它在某个**临界点**（critical point）达到它的下确界，即在梯度为零的点 $w^c$：
$$
\nabla E(w^c) := (\partial_1E(w^c), ..., \partial_dE(w^c)) = (0, ..., 0)
$$
这种**序列化方法**（sequential approach）的核心思想是构建一个序列 $(w_{(i)})_i$，使得 $\nabla E(w_{(i)})$ 逐步减小，并在 $\nabla E(w_{(i)})$ 足够接近零向量时停止过程。然后冻结得到的参数选择，这将生成模型 `Model(w)`，其经验误差接近于最低值。

选择越来越小的梯度误差参数的迭代过程称为**梯度下降**（gradient descent），其步骤如下：
1. 随机初始化一个参数选择 $w_{(0)} \in \mathbb{R}^d$。<br>
2. 计算该参数选择的经验误差的梯度 $\nabla E(w_{(0)})$。<br>
3. 通过在上一步参数选择的每个坐标上**减去**一部分误差来获得下一个参数选择：$w_{(1)} := w_{(0)} - \lambda \nabla E(w_{(0)})$。<br>
4. 重复这个过程，$w_{(i+1)} := w_{(i)} - \lambda \nabla E(w_{(i)})$，直到 $\nabla E(w_{(i)})$ 满足某个**停止准则**（stopping criterion）。<br>
5. 最后一个参数选择将输出用于定义最终的模型。<br>

其中，$\lambda > 0$ 称为**学习率**（learning rate）。

此外，另一个计算复杂性的来源是训练数据集本身的规模：在大型数据集上顺序地计算经验误差以找到更好的参数 $w^{*}$ 是不现实的。因此，**我们不再对整个数据集计算经验误差，而是逐个样本计算**。这一新的迭代过程称为**随机梯度下降**（stochastic gradient descent, SGD），其步骤如下：
1. 随机初始化一个参数选择 $w_{(0)} \in \mathbb{R}^d$。<br>
2. 随机打乱数据集 $D$，得到一个新的排序 $(x_1, y_1), ..., (x_n, y_n)$。<br>
3. 对于每个索引 $i \leq n$，计算误差函数 $E_i(w) := Error(Model(w), (x_i, y_i))$，并递归定义一个新的参数选择：$w_{(i+1)} := w_{(i)} - \lambda \nabla E_{i+1}(w_{(i)})$。<br>
4. 从步骤2重新开始，直到 $\nabla E_{i+1}(w_{(i)})$ 满足某个**停止准则**。<br>
5. 最后一个参数选择将输出用于定义最终的模型。<br>

学习率（learning rate）$\lambda$ 的典型值为 0.01，0.001 或更小。

对整个数据集 $D$ 进行一次完整的处理（即执行步骤 2 和 3）称为一个**周期**（epoch）。在一个周期之后，参数选择已被更新 $n$ 次，因此有时只需执行一个周期就可以满足停止准则。由于计算样本的梯度 $\nabla E_{i}$ 的计算成本远低于计算整个数据集的梯度 $\nabla E$，因此这种训练在实际中是可行的。

### Machine Learning in practice : Linear Regression

让我们使用数据科学中最常见的回归模型：**线性回归**（Linear regression）。线性回归是一个机器学习模型，它试图将目标变量表示为**解释变量的线性组合**（linear combination of the explanatory variables）。更具体地说，给定解释变量 $X^1, ..., X^p$，线性模型由 $(w_0, w_1, ..., w_p) \in \mathbb{R}^{p+1}$ 参数化，并定义为：
$$
\text{Regression\_model}(w_0, ..., w_p)\text{.predict}(x_1, ..., x_p) = w_0 + \sum_{i = 1}^p w_ix_i
$$

现在让我们创建并拟合（fit）这样一个模型：

In [2]:
Model = LinearRegression()

In [3]:
# 3 input variables 
# 2 data points
X = [
    [1, 2, 1],
    [-1, 0, 0.27],
]
y = [
    2, 
    2.5,
]

然后，学习阶段就通过嵌入在元模型中的 `fit` 方法简单地执行：

In [4]:
model = Model.fit(X, y)

恭喜！你现在拥有一个已经训练好的机器学习模型，可以用于推理了：

In [5]:
model.predict([[1, 2, 3]])

array([1.91444878])

In [6]:
2.3943559633887657 + (-0.11719345122994529)*1 + (-0.11719345122994528)*2 + (-0.04277560969893002)*3

1.9144487806021397

模型参数是通过以下方式获得的：

In [7]:
w_0 = model.intercept_
w_1, w_2, w_3 = model.coef_
w_0, w_1, w_2, w_3

(2.3943559633887657,
 -0.11719345122994529,
 -0.11719345122994528,
 -0.04277560969893002)

这是故事的结尾吗？嗯，还不完全是。

### Compute the learned parameters by hand

让我们考虑以下数据点集合 $D$，作为可解释值/目标值：
$$
(x_1, y_1) = (0, -1) \qquad \qquad \qquad (x_2, y_2) = (1, 1)
$$
我们的目标是计算由 $(w_0, w_1) \in \mathbb{R}^2$ 参数化的回归模型，定义为
$$
\text{Regression\_model}(w_0, w_1)\text{.predict}(x) = w_0 + w_1x
$$
使其在该数据集上最小化经验误差。

#### Exercice
1. 在平面上绘制数据集。猜测哪个参数选择是最优的，并提供一个可视化解释。
2. 找到经验误差函数 $E(w_0, w_1) := RMSError(\text{Regression\_model}(w_0, w_1), D)$ 的显式表达式。证明最小化 $E(w_0, w_1)$ 等价于最小化
$$
\tilde{E}(w_0, w_1) = (w_0 + 1)^2 + (w_0 + w_1 -1)^2
$$
3. 计算 $\tilde{E}$ 的梯度，并使用它找到该函数的全局最小值。
4. 计算从 $(w_{0, (0)}, w_{0, (1)}) = (1, 1)$ 开始，通过梯度下降计算出的参数序列的下4个项。将这些点和全局最小值一起在平面上绘制出来。
5. 计算随机梯度下降在2个epochs的结果，其中第一个epoch按数据集的原始顺序覆盖，第二个epoch按数据集的逆序覆盖。

### To go further

#### Variants of linear regression



在机器学习中，经常会遇到不同的线性回归变体，例如Ridge回归、Lasso回归和ElasticNet回归。它们都属于常规线性回归模型，因为它们的元模型与普通线性回归相同，并且在学习阶段后获得的模型表达式也是输入特征的加权和。每种变体的不同之处在于引入了**惩罚项**（penalisation term）来估计经验误差。显然，考虑不同的误差表达式最终会导致不同的最优参数，因此会产生不同的模型。我们提供了Ridge回归变体的经验风险公式（具体公式可能根据来源略有不同）：
$$
RidgeError(Model(\theta), D) = \frac{1}{n} \sqrt{\sum_{(x, y) \in D}(Model(\theta).predict(x) - y)^2 + \alpha \left(\sum_{\theta_i \in \theta} \theta_i^2 \right)}\\
LassoError(Model(\theta), D) = \frac{1}{n} \sqrt{\sum_{(x, y) \in D}(Model(\theta).predict(x) - y)^2 + \alpha \left(\sum_{\theta_i \in \theta} \vert \theta_i \vert \right)}\\
ElasticError(Model(\theta), D) = \frac{1}{n} \sqrt{\sum_{(x, y) \in D}(Model(\theta).predict(x) - y)^2 + \alpha_1 \left(\sum_{\theta_i \in \theta} \theta_i^2 \right) + \alpha_2 \left(\sum_{\theta_i \in \theta} \vert \theta_i \vert \right)}
$$
因此，模型参数向量的$L_1$或/和$L_2$范数被惩罚，使得在这种方案下的最优模型不仅在经验上能最大限度地进行近似，同时还能保持合理大小的参数。

更多关于这些变体的示例，可以参考[Scikit-Learn文档](https://scikit-learn.org/stable/modules/linear_model.html#linear-models)。